In [1]:
!pip install google-generativeai python-pptx Pillow requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 12.5 MB/s eta 0:00:00


In [2]:
import os
import google.generativeai as genai
from pptx import Presentation
from dotenv import load_dotenv
from pptx.util import Inches,Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import requests
from PIL import Image
import io
import json

load_dotenv()



False

In [27]:

class PPTGenerator:
    def __init__(self, api_key=None):
        self.api_key = api_key
        if not self.api_key:
            raise ValueError("Gemini API key not found.")

        genai.configure(api_key=self.api_key)

        # Using Gemini 2.5 Flash for high-speed generation
        self.model = genai.GenerativeModel('gemini-2.5-flash')
        self.presentation = Presentation()

    def generate_content_outline(self, topic, num_slides=5):
        prompt = f"""
        Create a detailed outline for a Powerpoint presentation on "{topic}" with {num_slides} slides.
        Return the response as a JSON array ONLY.
        Structure:
        [
          {{
            "title": "Slide Title",
            "content": "Bullet point 1. Bullet point 2.",
            "slide_type": "title|content|image|conclusion"
          }}
        ]
        """
        try:
            response = self.model.generate_content(prompt)
            content = response.text.strip()
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0].strip()
            elif "```" in content:
                content = content.split("```")[1].strip()
            return json.loads(content)
        except Exception as e:
            print(f"Error generating outline: {e}")
            return None

    def generate_image_description(self, slide_content):
        prompt = f"Create a 5-word image search query for: {slide_content}. Return only the phrase."
        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()
        except:
            return "professional business technology"

    def download_image(self, query, save_path="temp_image.jpg"):
        try:
            url = "https://api.pexels.com/v1/search"
            headers = {"Authorization": ""}  #provide your Pexels Api key to make it work
            params = {"query": query, "per_page": 1, "orientation": "landscape"}
            response = requests.get(url, headers=headers, params=params)
            data = response.json()
            if not data.get('photos'): return None
            image_url = data['photos'][0]['src']['large']
            image_res = requests.get(image_url)
            with open(save_path, 'wb') as f:
                f.write(image_res.content)
            return save_path
        except:
            return None

    def create_title_slide(self, title, subtitle="Created by Arka"):
        slide_layout = self.presentation.slide_layouts[0]
        slide = self.presentation.slides.add_slide(slide_layout)
        title_shape = slide.shapes.title
        title_shape.text = title

        title_para = title_shape.text_frame.paragraphs[0]
        title_para.font.size = Pt(44)
        title_para.font.bold = True
        title_para.font.color.rgb = RGBColor(44, 62, 80) # Slate Blue

        subtitle_shape = slide.placeholders[1]
        subtitle_shape.text = subtitle

    def create_content_slide(self, title, content, include_image=False):
        # Layout 1 is 'Title and Content'
        slide_layout = self.presentation.slide_layouts[1]
        slide = self.presentation.slides.add_slide(slide_layout)

        # 1. Style the Title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_frame = title_shape.text_frame.paragraphs[0]
        title_frame.font.size = Pt(32)
        title_frame.font.bold = True
        title_frame.font.color.rgb = RGBColor(44, 62, 80) # Dark Blue/Grey

        # 2. Prevent Overlap: Constraint Logic
        body_shape = slide.placeholders[1]
        if include_image:
            # Shift text to the left and shrink width
            body_shape.left = Inches(0.5)
            body_shape.top = Inches(1.6) # Lowered to avoid title overlap
            body_shape.width = Inches(4.5)
        else:
            body_shape.width = Inches(9)
            body_shape.top = Inches(1.5)

        # 3. Style the Text Body
        body_shape.text = content
        for paragraph in body_shape.text_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(60, 60, 60)
            paragraph.space_after = Pt(12) # Breathing room

        # 4. Right-Side Image Placement
        if include_image:
            img_desc = self.generate_image_description(content)
            img_path = self.download_image(img_desc)
            if img_path and os.path.exists(img_path):
                # Start image at 5.5 inches to ensure no collision
                slide.shapes.add_picture(img_path, Inches(5.5), Inches(1.8), width=Inches(4))
                os.remove(img_path)
        return slide

    def generate_presentation(self, topic, num_slides=5, output_file="presentation.pptx"):
        print(f"Starting generation for: {topic}")
        content_outline = self.generate_content_outline(topic, num_slides)
        if not content_outline: return

        for i, slide_data in enumerate(content_outline):
            title = slide_data.get('title', 'Untitled')
            content = slide_data.get('content', '')
            slide_type = slide_data.get('slide_type', 'content')

            if i == 0 or slide_type == "title":
                self.create_title_slide(title)
            else:
                # Every other slide or "image" slides get a split layout
                show_img = (i % 2 == 0) or (slide_type == "image")
                self.create_content_slide(title, content, include_image=show_img)

        self.presentation.save(output_file)
        print(f"Success! Saved as {output_file}")

In [28]:
api_key= "" #insert your gemini Api key


In [29]:
try:
  generator= PPTGenerator(api_key=api_key)
except ValueError as e:
  print(e)

In [30]:
topic= ""  # Whichever topic you want to make a ppt around
num_slides=6  # Number of slides you want

try:
  output_file = generator.generate_presentation(topic,num_slides,"Music.pptx")
except Exception as e:
  print(e)

Starting generation for: Music as a way to release stress
Success! Saved as Music.pptx
